# W207 Final Project
Erika, Jen Jen, Geoff, Leslie

(In Python 3)

As of 3/35

Outline:

* Data Pre-Processing  
* Simple Feature Selection
* Basline Models
* Possible Approaches
   

# Section 1 Loading and Processing Data

In [1]:
## Import Libraries ##
import json
from pprint import pprint
from pandas import *
from pandas.io.json import json_normalize


# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

/home/levi/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/levi/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
## Get Data ##

#reference on data: https://www.kaggle.com/c/random-acts-of-pizza/data
# pull in the training and test data
#with open('/Users/erikaananda/Documents/MIDS/W207/Final Project/data/train.json', encoding='utf-8') as data_file:
with open('/home/levi/Documents/W207_Proj/data/train.json', encoding='utf-8') as data_file:
    trainData = json.loads(data_file.read())   
#with open('/Users/erikaananda/Documents/MIDS/W207/Final Project/data/test.json', encoding='utf-8') as data_file:
with open('/home/levi/Documents/W207_Proj/data/train.json', encoding='utf-8') as data_file:
    testData = json.loads(data_file.read())    

# create a dev data set 
devData = trainData[0:1000]
trainData = trainData[1000:]

# show how the data looks in its original format
#pprint("data in json format:")
#pprint(trainData[1])

# create a normalized view
allTData = json_normalize(trainData)
print("\nSize of the normalized Data:", allTData.shape)
print("\nnormalized data columns:", list(allTData))

allDData = json_normalize(devData)



Size of the normalized Data: (3040, 32)

normalized data columns: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subreddits_at_request

In [4]:
## Create subsets of data for analysis ###

# create a flat dataset without the subreddits list
flatData = allTData.drop('requester_subreddits_at_request', 1)
# create a separate dataset with just subreddits, indexed on request id
# we can creata a count vector on the words, run Naive Bayes against it, 
# and add the probabilities to our flat dataset
subredTData = allTData[['request_id','requester_subreddits_at_request']]
subredTData.set_index('request_id', inplace=True)

subredDData= allDData[['request_id','requester_subreddits_at_request']]
subredDData.set_index('request_id', inplace=True)

# our training labels
trainLabel = allTData['requester_received_pizza']

devLabel = allDData['requester_received_pizza']

# what do these look like?
#print(list(flatData))
print(subredTData.shape)
#print(subredTData['requester_subreddits_at_request'][1])

# create a corpus of subreddits to vectorize
trainCorpus = []
for index in range(len(subredTData)):
    trainCorpus.append(' '.join(subredTData['requester_subreddits_at_request'][index]))

devCorpus = []
for index in range(len(subredDData)):
    devCorpus.append(' '.join(subredDData['requester_subreddits_at_request'][index]))
    


(3040, 1)


In [12]:
# combine all text sources into a single corpus
fldTText = allTData[['request_id','request_text', 'request_text_edit_aware', 'request_title']]
fldDText = allDData[['request_id','request_text', 'request_text_edit_aware', 'request_title']]

trainCorpus = []
for index in range(len(subredTData)):
    a = ' '.join(subredTData['requester_subreddits_at_request'][index])
    b = (a, fldTText['request_text'][index], fldTText['request_text_edit_aware'][index],
        fldTText['request_title'][index])
    trainCorpus.append(' '.join(b))

devCorpus = []
for index in range(len(subredDData)):
    a = ' '.join(subredDData['requester_subreddits_at_request'][index])
    b = (a, fldDText['request_text'][index], fldDText['request_text_edit_aware'][index],
         fldDText['request_title'][index])
    devCorpus.append(' '.join(b))

# Print 3 examples  
print (trainCorpus[:3])
labels = trainLabel.astype(int)
labels = list(labels)
print(labels[:3])
print('-'*75)

print ('\n' , devCorpus[:3])
labels_dev = devLabel.astype(int)
labels_dev = list(labels_dev)
print(labels_dev[:3])

["FoodstuffsAllAround IAmA RandomActsOfCookies RandomActsofCards RandomKindness Random_Acts_Of_Pizza comiccon cosplay cosplayers So it's been a while since it's happened, but yeah. Just got dumped by a girl I like... My brother has used up pretty much everything in the freezer. I would really appreciate a pizza right now... So it's been a while since it's happened, but yeah. Just got dumped by a girl I like... My brother has used up pretty much everything in the freezer. I would really appreciate a pizza right now... [Request] Just got dumped, no food in the freezer. Pizza?", "AskReddit Guitar Jazz Music NSFW_GIF Psychonaut RoomPorn StAugustine TwoXChromosomes WTF YouShouldKnow atheism aww bakedart catpictures cats crochet dubstep ents entwives food funny gonewild hiphopheads listentothis meetup offbeat pics realpics self sex tattoos treecomics treemusic trees videos vinyl zombies My boyfriend and I live in Saint Augustine, Florida and have been having a rough time financially the past

# Section 2. Simple Feature Selection and Pre-Processing

In [14]:
# Simple Pre-Processing

def data_preprocessor(s):
    """
    Note: this function pre-processors data:
    (1) removes non-alpha characters
    (2) converts digits to 'number'
    (3) regularizes spaces (although CountVectorizer ignores this unless they are part of words)
    (4) reduces word size to n
    """

    s = [re.sub(r'[?|$|.|!|@|\n|(|)|<|>|_|-|,|\']',r' ',s) for s in s] # strip out non-alpha numeric char, replace with space
    s = [re.sub(r'\d+',r'number ',s) for s in s] # convert digits to number
    s = [re.sub(r' +',r' ',s) for s in s] # convert multiple spaces to single space
    
    # This sets word size to n=5
    num = 5
    def set_word(s):
        temp = []
        for s in s:
            x = s.split()
            z = [elem[:num] for elem in x]
            z = ' '.join(z)
            temp.append(z)       
        return temp
    
    s = set_word(s)
    
    return s



In [24]:
# Set up the data with CountVectorizer

#vectorizer = CountVectorizer(lowercase=True, strip_accents='unicode',stop_words='english')

vectorizer = CountVectorizer(min_df=1,lowercase=True)
tVector = vectorizer.fit_transform(trainCorpus)
dVector = vectorizer.transform(devCorpus)

print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", tVector.shape[1])
print ("First 5 feature Names:", vectorizer.get_feature_names()[1:6], "\n")

tVector_p = vectorizer.fit_transform(data_preprocessor(trainCorpus))
dVector_p = vectorizer.transform(data_preprocessor(devCorpus))

print ('\nPre-Processed data:')
print ("The size of the vocabulary for the training text data is", tVector_p.shape[1])
print ("First 5 feature Names:", vectorizer.get_feature_names()[1:6], "\n")


Raw data:
The size of the vocabulary for the training text data is 17213
First 5 feature Names: ['000', '0000', '0011011001111000', '00243364', '00pm'] 


Pre-Processed data:
The size of the vocabulary for the training text data is 10491
First 5 feature Names: ['aaaaa', 'aan', 'ab', 'aback', 'aband'] 



# Section 3. Baseline Models

## Logistic Regression

In [43]:
# Logistic Regression

C = 0.01 #(For now)

clf = LogisticRegression(penalty='l2', C=C)
clf.fit(tVector,trainLabel)
logitScore = round(modelLogit.score(dVector, devLabel), 4)
print("For C = ", C, "Logistic regression accuracy:", logitScore)

clf.fit(tVector_p,trainLabel)
logitScore = round(modelLogit.score(dVector, devLabel), 4)
print("For C = ", C, "Logistic regression (processed data) accuracy:", logitScore)


For C =  0.01 Logistic regression accuracy: 0.741
For C =  0.01 Logistic regression (processed data) accuracy: 0.741


## Naive Bayes

In [50]:
# Multinomial NB

alpha = 0.01

clf = BernoulliNB(alpha=alpha)
clf.fit(tVector, trainLabel)
test_predicted_labels = clf.predict(dVector) 
print ('Bernoulli NB using raw data with alpha = %1.3f:' %alpha, metrics.accuracy_score(devLabel,test_predicted_labels) )

clf.fit(tVector_p, trainLabel)
test_predicted_labels = clf.predict(dVector_p) 
print ('Bernoulli NB using processed data  with alpha = %1.3f:' %alpha, metrics.accuracy_score(devLabel,test_predicted_labels) )


Bernoulli NB using raw data with alpha = 0.010: 0.719
Bernoulli NB using processed data  with alpha = 0.010: 0.708


## Logistic Regression More Feature Selection

In [40]:
# get the best regularization
regStrength = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 6.0, 10.0]
catCoef = []

for c in regStrength:
    modelLogit = LogisticRegression(penalty='l2', C=c)
    modelLogit.fit(tVector, trainLabel)
    logitScore = round(modelLogit.score(dVector, devLabel), 4)
    print("For C = ", c, "Logistic regression accuracy:", logitScore)

modelLogit = LogisticRegression(penalty='l2', C=.001)
modelLogit.fit(tVector, trainLabel)

print(max(modelLogit.coef_[0]))
numWeights = 25

sortIndex = np.argsort(modelLogit.coef_)
iLen = len(sortIndex[0])
print("\nTop", numWeights, "Weighted Features:")

for index in range((iLen - numWeights) , iLen):
    lookup = sortIndex[0][index]
    weight =  round(modelLogit.coef_[0][lookup], 5)
    print(vectorizer.get_feature_names()[sortIndex[0][index]], weight)

For C =  0.0001 Logistic regression accuracy: 0.74
For C =  0.001 Logistic regression accuracy: 0.741
For C =  0.01 Logistic regression accuracy: 0.736
For C =  0.1 Logistic regression accuracy: 0.703
For C =  0.5 Logistic regression accuracy: 0.689
For C =  1.0 Logistic regression accuracy: 0.689
For C =  2.0 Logistic regression accuracy: 0.683
For C =  6.0 Logistic regression accuracy: 0.675
For C =  10.0 Logistic regression accuracy: 0.674
0.0381825219695

Top 5 Weighted Features:
neofo 0.0286


IndexError: list index out of range

# Future Steps

* More data pre-processing (looking for newer features too)
* Explore PCA/LSA
* Ideas on features
    - Combination of words
    - Pruning
    